In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
import pandas as pd
import re
import os

def filter_data(input_file_path, output_file_path,keyword,affirm_report,order_report_table):
    
    for file in os.listdir(input_file_path):
        dataFrame= pd.read_csv(os.path.join(input_file_path, file))
        # select rows containing text "Affirm"
        data_filtered = dataFrame.loc[dataFrame['DESCRIPTION'].str.contains(keyword),['DATE', 'DESCRIPTION', 'RECEIVED']]
        # data_filtered.to_csv(os.path.join(output_file_path, file), index = False)

        #split the decription
        new =data_filtered.DESCRIPTION.str.split(expand=True)
        data_filtered['DEPOSIT_ID']=new[15]
        data_filtered.drop(columns =["DESCRIPTION"], inplace = True)
        data_filtered['DESCRIPTION']=new[3]+' '+new[4]

        # split the deposit id
        new_id=data_filtered.DEPOSIT_ID.str.split(":",expand=True)
        data_filtered.drop(columns =["DEPOSIT_ID"], inplace = True)
        data_filtered['DEPOSIT_ID']=new_id[1]
      
        #read the affirm report file
        data_report=pd.read_csv(affirm_report)
        data_report.rename(columns = {'deposit_id':'DEPOSIT_ID'}, inplace = True)
  
        #combine two csv files based on DEPOSIT ID
        df = pd.merge(data_filtered, data_report,on="DEPOSIT_ID")
        # print(df.to_string())
        df.rename(columns = {'RECEIVED':'BANK AMOUNT'}, inplace = True)
        df.rename(columns = {'order_id':'ORDER'}, inplace = True)
        df['FEE']=df['fees']+df['txn_fees']
        df['ORDER']=df['ORDER'].astype(str)

        #read the order report table
        report_table=pd.read_csv(order_report_table)
        report_table.rename(columns = {'Order_Number':'ORDER'}, inplace = True)
       
        #combine report table to df
        data_table=pd.merge(df,report_table,on='ORDER')
        data_table.rename(columns = {'Item_Price':'PRICE'}, inplace = True) # rename columns
        data_table.rename(columns = {'Item_Discount':'DISCOUNT'}, inplace = True)

        #add new columns
        data_table['NET AMOUNT']=None
        data_table['CHECK']=None

        print(data_table.to_string())
        # output_data=df[['DATE', 'DESCRIPTION','BANK AMOUNT','total_settled','ORDER','FEE']].copy()
        # print(output_data)
        #convert the output to csv file
        # output_data.to_csv('affirm_summary.csv', encoding = 'utf-8-sig') 
        # report='affirm_summary.csv'
        # output_data.to_csv(os.path.join(output_file_path,report), index = False) 
        
    

In [ ]:
filter_data('/content/drive/MyDrive/Colab Notebooks/input_file_path','/content/drive/MyDrive/Colab Notebooks/output_file_path','AFFIRM','/content/drive/MyDrive/Affirm Testing/Affirm_Report_0123.csv','/content/drive/MyDrive/Affirm Testing/order_report_Jan22_Jan23.csv')
